In [189]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
import nltk
import numpy as np
import pandas as pd
import os
import collections
import re
import string

In [139]:


prefix = 'v2/'
train_df = pd.read_csv(prefix + 'training_v2.tsv',  sep='\t', header=None)
dev_df = pd.read_csv(prefix + 'dev_v2.tsv', sep='\t', header=None)

train_df.columns = list(train_df.iloc[0])
train_df = train_df[1:]
dev_df.columns = list(dev_df.iloc[0])
dev_df = dev_df[1:]

train_df = pd.concat([train_df, dev_df])
train_df.reset_index(inplace = True) 

train_df

,index,topic_id,tweet_id,tweet_url,tweet_text,claim,check_worthiness
0,1,covid-19,1234964653014384644,https://twitter.com/EricTrump/status/123496465...,Since this will never get reported by the medi...,1,1
1,2,covid-19,1234869939720216578,https://twitter.com/RealJamesWoods/status/1234...,"Thanks, #MichaelBloomberg. Here’s a handy litt...",0,0
2,3,covid-19,1234873136304267267,https://twitter.com/hayxsmith/status/123487313...,"Folks, when you say ""The corona virus isn't a ...",0,0
3,4,covid-19,1235071285027147776,https://twitter.com/ipspankajnain/status/12350...,Just 1 case of Corona Virus in India and peop...,1,0
4,5,covid-19,1234911110861594624,https://twitter.com/PressSec/status/1234911110...,President @realDonaldTrump made a commitment...,1,1
...,...,...,...,...,...,...,...
817,146,covid-19,1235914080931766274,https://twitter.com/DrZweliMkhize/status/12359...,Stop spreading fake news #Coronavirus https://...,0,0
818,147,covid-19,1235770706765451264,https://twitter.com/globaltimesnews/status/123...,It's fake! It's fake!' shout residents of a co...,1,1
819,148,covid-19,1235973416995315712,https://twitter.com/wef/status/123597341699531...,Be Smart about #COVID19: 1️⃣ follow accurate p...,0,0
820,149,covid-19,1235675024738185239,https://twitter.com/999saudsalman/status/12356...,"On the left: @A_AlAthbah, a Qatari puppet, att...",1,1


In [144]:
eval_df = pd.read_csv('test-input.tsv', sep='\t', header=None)
eval_df.columns = list(eval_df.iloc[0])
eval_df = eval_df[1:]
eval_df.reset_index(inplace = True) 
eval_df

,index,topic_id,tweet_id,tweet_url,tweet_text
0,1,covid-19,1237160250513522688,https://twitter.com/user/status/12371602505135...,POTUS wanted everyone to know he was in close ...
1,2,covid-19,1237125962871037953,https://twitter.com/user/status/12371259628710...,Who would you prefer to lead our nation’s resp...
2,3,covid-19,1237207721604235264,https://twitter.com/user/status/12372077216042...,It was a really really really really really re...
3,4,covid-19,1237178597024108552,https://twitter.com/user/status/12371785970241...,Bald-faced #LIE. @replouiegohmert did self-qua...
4,5,covid-19,1237049051058561024,https://twitter.com/user/status/12370490510585...,LIVE: Daily media briefing on #COVID19 with @D...
...,...,...,...,...,...
135,136,covid-19,1237094362544209920,https://twitter.com/user/status/12370943625442...,This is how the Corona Virus is spreading and ...
136,137,covid-19,1237063585580318720,https://twitter.com/user/status/12370635855803...,Markets are crashing. Tourism is dying. Travel...
137,138,covid-19,1237159267070664705,https://twitter.com/user/status/12371592670706...,Pray for Palestine. State of Emergency. - 27 c...
138,139,covid-19,1236972990492897281,https://twitter.com/user/status/12369729904928...,ADVISORY TO STUDENTS: Class suspensions were m...


In [ ]:
print(train_df.shape)
print('\nclaim:\n',train_df.claim.value_counts())
print('\ncheck_worthiness:\n',train_df.check_worthiness.value_counts())

print('\n',eval_df.shape)
print('\nclaim:\n',eval_df.claim.value_counts())
print('\ncheck_worthiness:\n',eval_df.check_worthiness.value_counts())

In [146]:
import re
import string

def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

train_df['text_clean'] = train_df['tweet_text'].apply(lambda x: remove_punct(x))
eval_df['text_clean'] = eval_df['tweet_text'].apply(lambda x: remove_punct(x))
eval_df.head()

,index,topic_id,tweet_id,tweet_url,tweet_text,text_clean
0,1,covid-19,1237160250513522688,https://twitter.com/user/status/12371602505135...,POTUS wanted everyone to know he was in close ...,POTUS wanted everyone to know he was in close ...
1,2,covid-19,1237125962871037953,https://twitter.com/user/status/12371259628710...,Who would you prefer to lead our nation’s resp...,Who would you prefer to lead our nation’s resp...
2,3,covid-19,1237207721604235264,https://twitter.com/user/status/12372077216042...,It was a really really really really really re...,It was a really really really really really re...
3,4,covid-19,1237178597024108552,https://twitter.com/user/status/12371785970241...,Bald-faced #LIE. @replouiegohmert did self-qua...,Baldfaced LIE replouiegohmert did selfquaranti...
4,5,covid-19,1237049051058561024,https://twitter.com/user/status/12370490510585...,LIVE: Daily media briefing on #COVID19 with @D...,LIVE Daily media briefing on COVID19 with DrTe...


In [147]:
from nltk import word_tokenize

train_tokens = [word_tokenize(sen) for sen in train_df.text_clean]
eval_tokens = [word_tokenize(sen) for sen in eval_df.text_clean]

def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
train_lower_tokens = [lower_token(token) for token in train_tokens]
eval_lower_tokens = [lower_token(token) for token in eval_tokens]

In [201]:
from nltk.corpus import stopwords

stoplist = stopwords.words('english')

def removeStopWords(tokens): 
    return [word for word in tokens if word not in stoplist]

train_filtered_words = [removeStopWords(sen) for sen in train_lower_tokens]
train_df['Text_Final'] = [' '.join(sen) for sen in train_filtered_words]
train_df['tokens'] = train_filtered_words

eval_filtered_words = [removeStopWords(sen) for sen in eval_lower_tokens]
eval_df['Text_Final'] = [' '.join(sen) for sen in eval_filtered_words]
eval_df['tokens'] = eval_filtered_words

eval_df

,index,topic_id,tweet_id,tweet_url,tweet_text,text_clean,Text_Final,tokens
0,1,covid-19,1237160250513522688,https://twitter.com/user/status/12371602505135...,POTUS wanted everyone to know he was in close ...,POTUS wanted everyone to know he was in close ...,potus wanted everyone know close contact gaetz...,"[potus, wanted, everyone, know, close, contact..."
1,2,covid-19,1237125962871037953,https://twitter.com/user/status/12371259628710...,Who would you prefer to lead our nation’s resp...,Who would you prefer to lead our nation’s resp...,would prefer lead nation ’ response growing co...,"[would, prefer, lead, nation, ’, response, gro..."
2,3,covid-19,1237207721604235264,https://twitter.com/user/status/12372077216042...,It was a really really really really really re...,It was a really really really really really re...,really really really really really really real...,"[really, really, really, really, really, reall..."
3,4,covid-19,1237178597024108552,https://twitter.com/user/status/12371785970241...,Bald-faced #LIE. @replouiegohmert did self-qua...,Baldfaced LIE replouiegohmert did selfquaranti...,baldfaced lie replouiegohmert selfquarantine c...,"[baldfaced, lie, replouiegohmert, selfquaranti..."
4,5,covid-19,1237049051058561024,https://twitter.com/user/status/12370490510585...,LIVE: Daily media briefing on #COVID19 with @D...,LIVE Daily media briefing on COVID19 with DrTe...,live daily media briefing covid19 drtedros cor...,"[live, daily, media, briefing, covid19, drtedr..."
...,...,...,...,...,...,...,...,...
135,136,covid-19,1237094362544209920,https://twitter.com/user/status/12370943625442...,This is how the Corona Virus is spreading and ...,This is how the Corona Virus is spreading and ...,corona virus spreading fast spreads dont natur...,"[corona, virus, spreading, fast, spreads, dont..."
136,137,covid-19,1237063585580318720,https://twitter.com/user/status/12370635855803...,Markets are crashing. Tourism is dying. Travel...,Markets are crashing Tourism is dying Travel i...,markets crashing tourism dying travel reduced ...,"[markets, crashing, tourism, dying, travel, re..."
137,138,covid-19,1237159267070664705,https://twitter.com/user/status/12371592670706...,Pray for Palestine. State of Emergency. - 27 c...,Pray for Palestine State of Emergency 27 case...,pray palestine state emergency 27 cases corona...,"[pray, palestine, state, emergency, 27, cases,..."
138,139,covid-19,1236972990492897281,https://twitter.com/user/status/12369729904928...,ADVISORY TO STUDENTS: Class suspensions were m...,ADVISORY TO STUDENTS Class suspensions were ma...,advisory students class suspensions made safet...,"[advisory, students, class, suspensions, made,..."


In [202]:
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = " ".join([lemmatizer.lemmatize(i) for i in text])
    return lem_text

train_df['Text_Final'] = train_df['tokens'].apply(lambda x: word_lemmatizer(x))
eval_df['Text_Final'] = eval_df['tokens'].apply(lambda x: word_lemmatizer(x))

eval_df

,index,topic_id,tweet_id,tweet_url,tweet_text,text_clean,Text_Final,tokens
0,1,covid-19,1237160250513522688,https://twitter.com/user/status/12371602505135...,POTUS wanted everyone to know he was in close ...,POTUS wanted everyone to know he was in close ...,potus wanted everyone know close contact gaetz...,"[potus, wanted, everyone, know, close, contact..."
1,2,covid-19,1237125962871037953,https://twitter.com/user/status/12371259628710...,Who would you prefer to lead our nation’s resp...,Who would you prefer to lead our nation’s resp...,would prefer lead nation ’ response growing co...,"[would, prefer, lead, nation, ’, response, gro..."
2,3,covid-19,1237207721604235264,https://twitter.com/user/status/12372077216042...,It was a really really really really really re...,It was a really really really really really re...,really really really really really really real...,"[really, really, really, really, really, reall..."
3,4,covid-19,1237178597024108552,https://twitter.com/user/status/12371785970241...,Bald-faced #LIE. @replouiegohmert did self-qua...,Baldfaced LIE replouiegohmert did selfquaranti...,baldfaced lie replouiegohmert selfquarantine c...,"[baldfaced, lie, replouiegohmert, selfquaranti..."
4,5,covid-19,1237049051058561024,https://twitter.com/user/status/12370490510585...,LIVE: Daily media briefing on #COVID19 with @D...,LIVE Daily media briefing on COVID19 with DrTe...,live daily medium briefing covid19 drtedros co...,"[live, daily, media, briefing, covid19, drtedr..."
...,...,...,...,...,...,...,...,...
135,136,covid-19,1237094362544209920,https://twitter.com/user/status/12370943625442...,This is how the Corona Virus is spreading and ...,This is how the Corona Virus is spreading and ...,corona virus spreading fast spread dont natura...,"[corona, virus, spreading, fast, spreads, dont..."
136,137,covid-19,1237063585580318720,https://twitter.com/user/status/12370635855803...,Markets are crashing. Tourism is dying. Travel...,Markets are crashing Tourism is dying Travel i...,market crashing tourism dying travel reduced a...,"[markets, crashing, tourism, dying, travel, re..."
137,138,covid-19,1237159267070664705,https://twitter.com/user/status/12371592670706...,Pray for Palestine. State of Emergency. - 27 c...,Pray for Palestine State of Emergency 27 case...,pray palestine state emergency 27 case corona ...,"[pray, palestine, state, emergency, 27, cases,..."
138,139,covid-19,1236972990492897281,https://twitter.com/user/status/12369729904928...,ADVISORY TO STUDENTS: Class suspensions were m...,ADVISORY TO STUDENTS Class suspensions were ma...,advisory student class suspension made safety ...,"[advisory, students, class, suspensions, made,..."


In [203]:
#this is not used
pos = []                      
neg = []
for l in train_df.claim:
    if int(l) == 0:
        pos.append(0)
        neg.append(1)
    elif int(l) == 1:
        pos.append(1)
        neg.append(0)
        
train_df['claim_Pos']= pos
train_df['claim_Neg']= neg

train_claim_df = train_df[['Text_Final', 'tokens', 'claim', 'claim_Pos', 'claim_Neg']]

pos = []
neg = []


In [204]:
pos = []
neg = []
for l in train_df.check_worthiness:
    if int(l) == 0:
        pos.append(0)
        neg.append(1)
    elif int(l) == 1:
        pos.append(1)
        neg.append(0)
        
train_df['worth_Pos']= pos
train_df['worth_Neg']= neg

train_worth_df = train_df[['Text_Final', 'tokens', 'check_worthiness', 'worth_Pos', 'worth_Neg']]

train_worth_df

,Text_Final,tokens,check_worthiness,worth_Pos,worth_Neg
0,since never get reported medium wanted share c...,"[since, never, get, reported, media, wanted, s...",1,1,0
1,thanks michaelbloomberg ’ handy little uninten...,"[thanks, michaelbloomberg, ’, handy, little, u...",0,0,1
2,folk say corona virus isnt big deal kill disab...,"[folks, say, corona, virus, isnt, big, deal, k...",0,0,1
3,1 case corona virus india people crazy mask da...,"[1, case, corona, virus, india, people, crazy,...",0,0,1
4,president realdonaldtrump made commitment dona...,"[president, realdonaldtrump, made, commitment,...",1,1,0
...,...,...,...,...,...
817,stop spreading fake news coronavirus httpstcoh...,"[stop, spreading, fake, news, coronavirus, htt...",0,0,1
818,fake fake shout resident community covid19 epi...,"[fake, fake, shout, residents, community, covi...",1,1,0
819,smart covid19 1️⃣ follow accurate public healt...,"[smart, covid19, 1️⃣, follow, accurate, public...",0,0,1
820,left aalathbah qatari puppet attack saudi arab...,"[left, aalathbah, qatari, puppet, attacks, sau...",1,1,0


In [205]:
all_training_words = [word for tokens in train_worth_df["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in train_worth_df["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))


16287 words total, with a vocabulary size of 5741
Max sentence length is 52


In [206]:
all_test_words = [word for tokens in eval_df["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in eval_df["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

2979 words total, with a vocabulary size of 1674
Max sentence length is 34


In [207]:

MAX_SEQUENCE_LENGTH = 10000
EMBEDDING_DIM = 300

worth_df = pd.concat([eval_df, train_worth_df])

model = models.Word2Vec(sentences = worth_df['tokens'], size=EMBEDDING_DIM, window=5, workers=4, min_count=1)
filename = 'gensim_word2vec_model.txt'
model.wv.save_word2vec_format(filename, binary=False)

In [208]:
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [209]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [210]:
training_embeddings = get_word2vec_embeddings(word2vec, train_worth_df, generate_missing=True)

MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 300

In [211]:

tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(train_worth_df["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(train_worth_df["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 5392 unique tokens.


In [212]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [213]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(5393, 300)


In [214]:
test_sequences = tokenizer.texts_to_sequences(eval_df["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [215]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [216]:
label_names = ['worth_Pos', 'worth_Neg']

In [217]:
y_train = train_worth_df[label_names].values

In [218]:
x_train = train_cnn_data
y_tr = y_train

In [219]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1000, 300)    1617900     input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 999, 200)     120200      embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 998, 200)     180200      embedding_3[0][0]                
____________________________________________________________________________________________

In [220]:
num_epochs = 10
batch_size = 32

In [221]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Train on 739 samples, validate on 83 samples
Epoch 1/10
739/739 [==============================] - 51s 70ms/step - loss: 0.6602 - acc: 0.6455 - val_loss: 0.7681 - val_acc: 0.4940
Epoch 2/10
739/739 [==============================] - 45s 61ms/step - loss: 0.5226 - acc: 0.7300 - val_loss: 0.7804 - val_acc: 0.5241
Epoch 3/10
739/739 [==============================] - 48s 65ms/step - loss: 0.4271 - acc: 0.8153 - val_loss: 0.6140 - val_acc: 0.6928
Epoch 4/10
739/739 [==============================] - 47s 64ms/step - loss: 0.2844 - acc: 0.9060 - val_loss: 0.6602 - val_acc: 0.6145
Epoch 5/10
739/739 [==============================] - 46s 63ms/step - loss: 0.1854 - acc: 0.9452 - val_loss: 0.5863 - val_acc: 0.6566
Epoch 6/10
739/739 [==============================] - 46s 63ms/step - loss: 0.2036 - acc: 0.9242 - val_loss: 0.9642 - val_acc: 0.5843
Epoch 7/10
739/739 [==============================] - 46s 63ms/step - loss: 0.0722 - acc: 0.9824 - val_loss: 0.7998 - val_acc: 0.6386
Epoch 8/10
739/73

In [222]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

predictions

140/140 [==============================] - 4s 26ms/step


array([[4.04229760e-03, 9.94853616e-01],
       [4.48756963e-01, 5.02907574e-01],
       [6.91384077e-03, 9.89889860e-01],
       [2.38162875e-02, 9.59601164e-01],
       [5.39456887e-05, 9.99917865e-01],
       [9.57050920e-03, 9.86539960e-01],
       [3.76641542e-01, 6.09090626e-01],
       [8.18279982e-01, 1.99079454e-01],
       [3.13401222e-03, 9.95075583e-01],
       [1.94732010e-01, 8.19040656e-01],
       [3.50572228e-01, 6.71010911e-01],
       [7.07852364e-01, 2.65340984e-01],
       [9.74031568e-01, 3.78495753e-02],
       [2.61447281e-01, 6.89359665e-01],
       [2.04459369e-01, 7.72952318e-01],
       [2.31613189e-01, 7.02329934e-01],
       [4.98584211e-02, 9.45553899e-01],
       [9.74889457e-01, 2.44410336e-02],
       [1.79822624e-01, 7.57784963e-01],
       [5.17639577e-01, 4.10648227e-01],
       [8.57847929e-02, 9.06406045e-01],
       [2.10646749e-01, 8.18648875e-01],
       [2.20333040e-02, 9.80277300e-01],
       [3.41748118e-01, 7.16951609e-01],
       [7.845641

In [181]:
labels = [1, 0]

In [182]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])
    
print(len(prediction_labels))
prediction_labels

#eval_worth_df.check_worthiness

140


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0]

In [225]:
#this was used to check performace of model on the dev set
#sum(eval_worth_df.check_worthiness==prediction_labels)/len(prediction_labels)
#print(eval_worth_df.check_worthiness==prediction_labels)

In [ ]:
eval_worth_df.check_worthiness.value_counts()

In [ ]:
from sklearn.metrics import classification_report

truth = []
for l in eval_worth_df.check_worthiness:
    truth.append(int(l))


print(classification_report(truth, prediction_labels))

In [ ]:
predictions[:,0]

In [223]:
preds_df = pd.DataFrame(predictions[:,0])
preds_df.columns = ['predictions']

In [228]:
submit = pd.merge(eval_df, preds_df, left_index=True, right_index=True)
submit

,index,topic_id,tweet_id,tweet_url,tweet_text,text_clean,Text_Final,tokens,predictions
0,1,covid-19,1237160250513522688,https://twitter.com/user/status/12371602505135...,POTUS wanted everyone to know he was in close ...,POTUS wanted everyone to know he was in close ...,potus wanted everyone know close contact gaetz...,"[potus, wanted, everyone, know, close, contact...",0.004042
1,2,covid-19,1237125962871037953,https://twitter.com/user/status/12371259628710...,Who would you prefer to lead our nation’s resp...,Who would you prefer to lead our nation’s resp...,would prefer lead nation ’ response growing co...,"[would, prefer, lead, nation, ’, response, gro...",0.448757
2,3,covid-19,1237207721604235264,https://twitter.com/user/status/12372077216042...,It was a really really really really really re...,It was a really really really really really re...,really really really really really really real...,"[really, really, really, really, really, reall...",0.006914
3,4,covid-19,1237178597024108552,https://twitter.com/user/status/12371785970241...,Bald-faced #LIE. @replouiegohmert did self-qua...,Baldfaced LIE replouiegohmert did selfquaranti...,baldfaced lie replouiegohmert selfquarantine c...,"[baldfaced, lie, replouiegohmert, selfquaranti...",0.023816
4,5,covid-19,1237049051058561024,https://twitter.com/user/status/12370490510585...,LIVE: Daily media briefing on #COVID19 with @D...,LIVE Daily media briefing on COVID19 with DrTe...,live daily medium briefing covid19 drtedros co...,"[live, daily, media, briefing, covid19, drtedr...",0.000054
...,...,...,...,...,...,...,...,...,...
135,136,covid-19,1237094362544209920,https://twitter.com/user/status/12370943625442...,This is how the Corona Virus is spreading and ...,This is how the Corona Virus is spreading and ...,corona virus spreading fast spread dont natura...,"[corona, virus, spreading, fast, spreads, dont...",0.303360
136,137,covid-19,1237063585580318720,https://twitter.com/user/status/12370635855803...,Markets are crashing. Tourism is dying. Travel...,Markets are crashing Tourism is dying Travel i...,market crashing tourism dying travel reduced a...,"[markets, crashing, tourism, dying, travel, re...",0.038802
137,138,covid-19,1237159267070664705,https://twitter.com/user/status/12371592670706...,Pray for Palestine. State of Emergency. - 27 c...,Pray for Palestine State of Emergency 27 case...,pray palestine state emergency 27 case corona ...,"[pray, palestine, state, emergency, 27, cases,...",0.458088
138,139,covid-19,1236972990492897281,https://twitter.com/user/status/12369729904928...,ADVISORY TO STUDENTS: Class suspensions were m...,ADVISORY TO STUDENTS Class suspensions were ma...,advisory student class suspension made safety ...,"[advisory, students, class, suspensions, made,...",0.002989


In [ ]:
df.drop(['tweet_url','tweet_text',''], axis = 1)